In [3]:
import pandas as pd
import numpy as np 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.read_excel('HIST_PAINEL_COVIDBR_20200515.xlsx')

In [4]:
a = df[['estado', 'municipio', 'data']].groupby(['estado', 'municipio']).agg(np.count_nonzero)
# a.loc['Uberlândia']
# a.sort_values('data', ascending=False)
# a.index

In [5]:
b = df[df.municipio.notna()][['municipio', 'data']].groupby(['data']).agg(np.count_nonzero)
# b.index
# df[df.municipio.notna()].data

In [6]:
df2 = pd.DataFrame(columns=b.index, index = a.index)
# df2

In [8]:
for uf_mun in df2.index:
    c = df[(df.estado == uf_mun[0]) & (df.municipio == uf_mun[1])][['data', 'obitosAcumulado']].set_index('data')
    for data in c.index:
        df2.at[(uf_mun[0], uf_mun[1]), data] = c.loc[data].obitosAcumulado
    # print(uf_mun[0], uf_mun[1])

In [9]:
# c.loc['2020-03-29'].obitosAcumulado
# df2.iloc[0]
# df2 = df2.fillna(0)
# df2.loc[('SP', 'São Paulo')]
# df2

In [22]:
tam = len(df2.columns)
df3 = df2.iloc[:,1:tam]
df3.columns = list(range(0,tam-1))
df4 = df2.iloc[:,0:tam-1]
df4.columns = list(range(0,tam-1))
df5 = df3 - df4
df5.columns = df2.columns[1:]
# df5.head(10)

In [38]:
# df5.loc[('SP', 'São Paulo')]
df5 = df5.fillna(0)
df6 = df5.copy()
for x in df5.index:
    # print(x)
    for c in range(0,tam):
        # print(c)
        if c >= 7:
            # print(f'{df.columns[c]:<15} {df3.loc[x][c-7:c].mean():10,.0f}')
            df6.at[x, df2.columns[c]] = df5.loc[x][c-7:c].mean()
# df6 = df6.sort_values('5/11/20', ascending=False)
# df6.iloc[:10,-14:]

In [39]:
q_pico = []
d_pico = []
for x in df6.index:
    # print(f'{x[0]:<2} {x[1]:<20} {df6.loc[x].max():5} {df6.columns[df6.loc[x].argmax()]}')
    q_pico.append(df6.loc[x].max())
    d_pico.append(df6.columns[df6.loc[x].argmax()])

In [40]:
df7 = pd.DataFrame(df6.iloc[:,-1], index=df6.index)
df7['Q_Pico'] = pd.Series(q_pico, index=df6.index)
df7['D_Pico'] = pd.Series(d_pico, index=df6.index)
df7.columns = ['Ult_7_Dias', 'Pico', 'Dia_do_Pico']
df7['Delta %'] = df7['Ult_7_Dias'] / df7['Pico'] - 1
df7 = df7.sort_values('Ult_7_Dias', ascending=False)
df7.head(10)

,,Ult_7_Dias,Pico,Dia_do_Pico,Delta %
estado,municipio,,,,
RJ,Rio de Janeiro,89,89,2020-05-15,0.00
SP,São Paulo,80,94,2020-04-29,-0.15
CE,Fortaleza,49,53,2020-05-13,-0.08
PA,Belém,47,51,2020-05-14,-0.08
AM,Manaus,40,41,2020-05-09,-0.02
PE,Recife,22,22,2020-05-15,0.00
MA,São Luís,15,15,2020-05-11,0.00
PA,Ananindeua,9,10,2020-05-14,-0.10
SP,Osasco,8,9,2020-05-12,-0.11


### Número de óbtidos por COVID-19
    Ranking 10 maiores cidades brasileiras - considerando últ 7 dias
    Valores apresentam média diária de óbitos em 7 dias
    Atualizado com dados até 15/05/2020
    Data no formato aaaa-mm-dd
    Fonte: Secretarias Estaduais de Saúde. Brasil.
    https://covid.saude.gov.br/